In [ ]:
# Cell 1: Import libraries and set up directories
import os
import cv2
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# # Cell 2: Extract frames from videos
# def extract_frames(video_path, output_folder, label):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
    
#     cap = cv2.VideoCapture(video_path)
#     frame_count = 0
    
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         frame_name = os.path.join(output_folder, f"{label}_{frame_count}.jpg")
#         cv2.imwrite(frame_name, frame)
#         frame_count += 1
    
#     cap.release()
#     cv2.destroyAllWindows()

# video_folders = {
#     "violence": "Dataset/Violence",
#     "non_violence": "Dataset/NonViolence"
# }

# output_folder = "extract"

# for label, folder in video_folders.items():
#     for video_file in os.listdir(folder):
#         video_path = os.path.join(folder, video_file)
#         extract_frames(video_path, os.path.join(output_folder, label), label)


In [ ]:
# Cell 3: Extract frames from videos
def prepare_yolo_data(frame_folder, labels):
    image_files = []
    for label in labels:
        label_folder = os.path.join(frame_folder, label)
        for file in os.listdir(label_folder):
            if file.endswith(".jpg"):
                image_files.append((os.path.join(label_folder, file), labels.index(label)))

    df = pd.DataFrame(image_files, columns=["image_path", "label"])
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"])

    def save_labels(df, split):
        os.makedirs(f"data/{split}/images", exist_ok=True)
        os.makedirs(f"data/{split}/labels", exist_ok=True)

        for _, row in df.iterrows():
            image_path, label = row["image_path"], row["label"]
            base_name = os.path.basename(image_path)
            new_image_path = f"data/{split}/images/{base_name}"
            label_path = new_image_path.replace("images", "labels").replace(".jpg", ".txt")

            if os.path.exists(new_image_path):
                print(f"File {new_image_path} already exists. Skipping.")
                continue
            try:
                shutil.move(image_path, new_image_path)
            except FileExistsError:
                print(f"File {new_image_path} already exists. Skipping.")
                continue
            with open(label_path, "w") as f:
                f.write(f"{label} 0.5 0.5 1 1\n")

    save_labels(train_df, "train")
    save_labels(val_df, "val")

prepare_yolo_data("extract", ["non_violence", "violence"])

In [ ]:
# Cell 4: Save data configuration
data_config = {
    'path': 'data',  
    'train': 'train/images', 
    'val': 'val/images',  
    'names': ['non_violence', 'violence']  
}

import yaml

with open('data_config.yaml', 'w') as f:
    yaml.dump(data_config, f)

In [ ]:
# Cell 5: Train YOLOv8 model
from ultralytics import YOLO
model = YOLO('Models/yolov8n.pt') 
model.train(data='data_config.yaml', epochs=25, imgsz=640)
model.save('Models/Violence.pt')